# 线性回归笔记

## 1、pytorch中的广播机制

pytorch中的广播机制和numpy中的广播机制一样, 因为都是数组的广播机制。

### 核心
如果相加的两个数组的shape不同, 就会触发广播机制, 
1)程序会自动执行操作使得A.shape==B.shape, 
2)对应位置进行相加,运算结果的shape是:A.shape和B.shape对应位置的最大值
比如:A.shape=(1,9,4),B.shape=(15,1,4),那么A+B的shape是(15,9,4)

两个维度不同的Tensor可以相乘, 示例：

In [6]:
a = torch.arange(0, 6).reshape((6,))
b = torch.arange(0, 12).reshape((2, 6))
# a和b的ndim不同, 但是可以element-wise相乘, 因为用到了广播机制
res = torch.mul(a, b)
print(f'a={a}\na.shape={a.shape}\na.ndim={a.ndim}')
print(f'b={b}\nb.shape={b.shape}\nb.ndim={b.ndim}')
print(f'res={res}\nres.shape={res.shape}\nres.ndim={res.ndim}')

a=tensor([0, 1, 2, 3, 4, 5])
a.shape=torch.Size([6])
a.ndim=1
b=tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])
b.shape=torch.Size([2, 6])
b.ndim=2
res=tensor([[ 0,  1,  4,  9, 16, 25],
        [ 0,  7, 16, 27, 40, 55]])
res.shape=torch.Size([2, 6])
res.ndim=2


### 有两种情况能够进行广播
1、A.ndim > B.ndim, 并且A.shape最后几个元素包含B.shape, 比如下面三种情况:
	A.shape=(2,3,4,5), B.shape=(3,4,5)
	A.shape=(2,3,4,5), B.shape=(4,5)
	A.shape=(2,3,4,5), B.shape=(5)
2、A.ndim == B.ndim, 并且A.shape和B.shape对应位置的元素要么相同要么其中一个是1, 比如
	A.shape=(1,9,4), B.shape=(15,1,4)
	A.shape=(1,9,4), B.shape=(15,1,1)
注意不要混淆ndim和shape这两个基本概念，基本上ndim=len(shape)

#### A.ndim > B.ndim

In [11]:
import numpy as np
a = np.arange(1, 49).reshape((2, 2, 3, 4))
b = np.arange(1, 13).reshape((3, 4))
# numpy会将b.shape调整至(2,2,3,4), 这一步相当于numpy自动实现np.tile(b,[2,2,1,1])
res = a + b
print('a', a.shape)
print('b', b.shape)
print('a+b', res.shape)
print(a+b == a + np.tile(b, [2, 2, 1, 1]))


a (2, 2, 3, 4)
b (3, 4)
a+b (2, 2, 3, 4)
[[[[ True  True  True  True]
   [ True  True  True  True]
   [ True  True  True  True]]

  [[ True  True  True  True]
   [ True  True  True  True]
   [ True  True  True  True]]]


 [[[ True  True  True  True]
   [ True  True  True  True]
   [ True  True  True  True]]

  [[ True  True  True  True]
   [ True  True  True  True]
   [ True  True  True  True]]]]


#### A.ndim == B.ndim

In [12]:
a = np.arange(12).reshape(4,3)
b = np.arange(4).reshape(4,1)
# numpy会将b.shape调整至(4,3), 这一步相当于numpy自动实现np.tile(b,[1,3])
res = a + b
print('a', a.shape)
print('b', b.shape)
print('a+b', res.shape)
print((a+b == a + np.tile(b,[1,3])) )

a (4, 3)
b (4, 1)
a+b (4, 3)
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]]


In [13]:
a = np.arange(1,37).reshape((1,9,4))
b = np.arange(1,61).reshape((15,1,4))
res = a + b
print('a', a.shape)
print('b', b.shape)
print('a+b', res.shape)
q = np.tile(a,[15,1,1]) + np.tile(b,[1,9,1])
print(q == res)

a (1, 9, 4)
b (15, 1, 4)
a+b (15, 9, 4)
[[[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]]

 [[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]]

 [[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]]

 [[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  

## 2、参数的形状与批量大小没有关系
正是因为如此，对同一个模型，我们可以选择不同的批量大小。

## 3、View()函数
在torch.Tensor.view()下的一个函数，可以有tensor调用，也可以有variable调用。

其作用在于返回和原tensor数据个数相同，但size不同的tensor

【Numpy中的size是元素个数，但是在Pytorch中size等价为Numpy中的shape】

view函数的-1参数的作用在于基于另一参数，自动计算该维度的大小

In [19]:
import torch
x = torch.arange(3)
y1 = x.view(3, 1)
y2 = x.view(y1.size())
y3 = x.view(3)
y4 = x.view(-1)
y5 = x.view(-1,1)
print(x)
print(y1)
print(y2)
print(y3)
print(y4)
print(y5)

tensor([0, 1, 2])
tensor([[0],
        [1],
        [2]])
tensor([[0],
        [1],
        [2]])
tensor([0, 1, 2])
tensor([0, 1, 2])
tensor([[0],
        [1],
        [2]])
